In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy import stats
import os
import math
import cv2
import PIL
from PIL import Image  
%matplotlib inline
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
       
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]  
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [ ]:
os.listdir("Test_Images/")

In [ ]:
def change_color_space(img, color_space='HSV'):
    space = cv2.COLOR_RGB2HSV
    if color_space == 'HLS':
        space = cv2.COLOR_RGB2HLS
    if color_space == 'LAB':
        space = cv2.COLOR_RGB2LAB
    if color_space == 'RGB':
        space = None
    if space is not None:
        img_copy = cv2.cvtColor(img, space)
    else:
        img_copy = img     
    return img_copy

In [ ]:
for name_img in os.listdir("Test_Images/"):
    image = mpimg.imread('Test_Images/'+name_img)
    plt.figure(figsize=(10,12))
    plt.subplot(1,4,1)
    plt.imshow(image)
    plt.title('original RGB')
    img = change_color_space(image, 'HSV')
    plt.subplot(1,4,2)
    plt.imshow(img)
    plt.title('HSV')
    img = change_color_space(image,'HLS')
    plt.subplot(1,4,3)
    plt.imshow(img)
    plt.title('HLS')
    img = change_color_space(image,'LAB')
    plt.subplot(1,4,4)
    plt.imshow(img)
    plt.title('LAB')
plt.show()

In [ ]:
for name_img in os.listdir("Test_Images/"):
    image = mpimg.imread('Test_Images/'+name_img)
    plt.figure(figsize=(10,12))
    plt.subplot(1,4,1)
    plt.imshow(image)
    new_img = change_color_space(image, 'HSV')
    for channel in range(3):
        isolated_channel_image = new_img[:,:,channel]
        plt.subplot(1,4,channel+2)
        plt.imshow(isolated_channel_image, cmap='gray')
    plt.show()

In [ ]:
for name_img in os.listdir("Test_Images/"):
    image = mpimg.imread('Test_Images/'+name_img)
    plt.figure(figsize=(10,12))
    plt.subplot(1,4,1)
    plt.imshow(image)
    new_img = change_color_space(image, 'HLS')
    for channel in range(3):
        isolated_channel_image = new_img[:,:,channel]
        plt.subplot(1,4,channel+2)
        plt.imshow(isolated_channel_image, cmap='gray')
    plt.show()

In [ ]:
for name_img in os.listdir("Test_Images/"):
    image = mpimg.imread('Test_Images/'+name_img)
    plt.figure(figsize=(10,12))
    plt.subplot(1,4,1)
    plt.imshow(image)
    new_img = change_color_space(image, 'LAB')
    for channel in range(3):
        isolated_channel_image = new_img[:,:,channel]
        plt.subplot(1,4,channel+2)
        plt.imshow(isolated_channel_image, cmap='gray')
    plt.show()
    

In [ ]:
images={}
for index, name_img in enumerate(os.listdir("Test_Images/")):
    image = mpimg.imread('Test_Images/'+name_img)
    if image.shape[2]>3:
        image = image[:,:,:3]
    LAB_img = change_color_space(image,'LAB')
    images[index]={'original': image, 'LAB': LAB_img, 'B_channel': LAB_img[:,:,2]}
print('number of LAB images: ', len(LAB_img))

In [ ]:
def extract_line(img, lower, upper):
    mask = cv2.inRange(img, np.array(lower,dtype=np.uint8), np.array(upper, dtype=np.uint8))
    return mask

In [ ]:
for index in range(len(images)):
    img = images[index]['original']
    LAB_img = images[index]['LAB']
    B_ch = images[index]['B_channel']
    plt.figure(figsize=(12,12))
    plt.subplot(1,4,1)
    plt.imshow(img)
    plt.title('original RGB')
    plt.subplot(1,4,2)
    plt.imshow(B_ch)
    plt.title('B channel LAB color space')
    yellow_lines = extract_line(LAB_img, [100,100,150], [220,180,255])
    plt.subplot(1,4,3)
    plt.imshow(yellow_lines, cmap = 'gray')
    plt.title('Extracted Yellow lines')
    hough_img, lines = hough_lines(yellow_lines, rho=1, theta=np.pi/180, threshold=50, min_line_len=10, max_line_gap=10)
    plt.subplot(1,4,4)
    plt.imshow(hough_img, cmap = 'gray')
    plt.title('Reconstructed Hough Yellow lines') 
    images[index]['yellow_mask'] = yellow_lines
    images[index]['hough_image_y'] = hough_img
    images[index]['hough_yellow_lines'] = lines   
plt.show()

In [ ]:
for index, name_img in enumerate(os.listdir("Test_Images/")):
    assert len(os.listdir("Test_Images/"))==len(images), " "
    image = mpimg.imread('Test_Images/'+name_img)
    HLS_img = change_color_space(image,'HLS')
    images[index]['HLS'] = HLS_img
print('number of HLS images: ', len(images))

In [ ]:
for index in range(len(images)):
    img = images[index]['original']
    HLS_img = images[index]['HLS']
    plt.figure(figsize=(12,12))
    plt.subplot(1,4,1)
    plt.imshow(img)
    plt.title('Original RGB')
    white_lines = extract_line(HLS_img, [0,200,0], [180,255,255])
    plt.subplot(1,4,2)
    plt.imshow(white_lines, cmap = 'gray')
    plt.title('Extracted White lines')
    white_lines = cv2.dilate(white_lines, kernel=np.ones((5,5), np.uint8), iterations=2)
    white_lines = cv2.erode(white_lines, kernel=np.ones((5,5), np.uint8), iterations=2)
    white_lines = cv2.dilate(white_lines, kernel=np.ones((5,5), np.uint8), iterations=1)
    plt.subplot(1,4,3)
    plt.imshow(white_lines, cmap = 'gray')
    plt.title('Enhanced White lines')  
    hough_img, lines = hough_lines(white_lines, rho=1, theta=np.pi/180, threshold=5, min_line_len=3, max_line_gap=30)
    plt.subplot(1,4,4)
    plt.imshow(hough_img, cmap = 'gray')
    plt.title('Reconstructed Hough White lines') 
    images[index]['white_mask'] = white_lines
    images[index]['hough_image_w'] = hough_img
    images[index]['hough_white_lines'] = lines   
plt.show()

In [ ]:
for index in range(len(images)):
    img = images[index]['original']
    yellow_lines = images[index]['yellow_mask']
    white_lines = images[index]['white_mask']
    plt.figure(figsize=(8,10))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title('original RGB')
    line_mask = yellow_lines + white_lines
    masked_img = np.copy(img)
    masked_img = cv2.dilate(masked_img, kernel=np.ones((5,5), np.uint8), iterations=2)
    masked_img = cv2.erode(masked_img, kernel=np.ones((5,5), np.uint8), iterations=2)
    masked_img = cv2.dilate(masked_img, kernel=np.ones((5,5), np.uint8), iterations=1)
    masked_img[line_mask!=255] = [0,0,0]
    plt.subplot(1,2,2)
    plt.imshow(masked_img)
    plt.title('Original after lane detection')
    images[index]['masked'] = masked_img
plt.show()

In [ ]:
def get_vertices_for_img(img):
    imshape = img.shape
    height = imshape[0]
    width = imshape[1]
    vert = None
    
    if (width, height) == (960, 540):
        region_bottom_left = (130 ,imshape[0] - 1)
        region_top_left = (410, 330)
        region_top_right = (650, 350)
        region_bottom_right = (imshape[1] - 30,imshape[0] - 1)
        vert = np.array([[region_bottom_left , region_top_left, region_top_right, region_bottom_right]], dtype=np.int32)
    else:
        region_bottom_left = (200 , 680)
        region_top_left = (600, 450)
        region_top_right = (750, 450)
        region_bottom_right = (1100, 650)
        vert = np.array([[region_bottom_left , region_top_left, region_top_right, region_bottom_right]], dtype=np.int32)
    return vert

def region_of_interest(img):
    mask = np.zeros_like(img)   
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255    
    vert = get_vertices_for_img(img)    
    cv2.fillPoly(mask, vert, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [ ]:
for index in range(len(images)):
    img = images[index]['original']
    masked = images[index]['masked']
    plt.figure(figsize=(15,15))
    plt.subplot(1,4,1)
    plt.imshow(img)
    plt.title('original RGB')
    plt.subplot(1,4,2)
    plt.imshow(masked)
    plt.title('original after lane extraction')
    cleaned_img = region_of_interest(masked)
    plt.subplot(1,4,3)
    plt.imshow(cleaned_img)
    plt.title('original extraction cleaned')
    hough_img, lines = hough_lines(grayscale(cleaned_img), rho=1, theta=np.pi/180, threshold=17, min_line_len=7, max_line_gap=0)
    plt.subplot(1,4,4)
    plt.imshow(hough_img)
    plt.title('Hough reconstruction')
    images[index]['cleaned'] = cleaned_img
    images[index]['hough_lines'] = lines
    plt.show()

In [ ]:
for index in range(len(images)):
    img = images[index]['original']
    cleaned = images[index]['cleaned']
    lines = images[index]['hough_lines']
    img_with_lines = weighted_img(cleaned, img, α=0.8, β=1., γ=0.) 
    copy_img = np.copy(img)
    draw_lines(copy_img, lines, color=[255, 0, 0], thickness=2) 
    plt.figure(figsize=(8,10))
    plt.subplot(1,3,1)
    plt.imshow(img)
    plt.title('original RGB')
    plt.subplot(1,3,2)
    plt.imshow(img_with_lines)
    plt.title('overlaid with lane mask')
    plt.subplot(1,3,3)
    plt.imshow(copy_img)
    plt.title('overlaid with hough lines')   
    images[index]['with_lines'] = img_with_lines
    plt.show()

In [ ]:
def find_lane_lines_formula(lines):
    xs = []
    ys = [] 
    for line in lines:
        for x1, y1, x2, y2 in line:
            xs.append(x1)
            xs.append(x2)
            ys.append(y1)
            ys.append(y2)  
    slope, intercept, r_value, p_value, std_err = stats.linregress(xs, ys)   
    return (slope, intercept)

In [ ]:
def draw_full_lines(img, slope, intersept, color, thickness):
    y = np.array([int(img.shape[0]*0.63),img.shape[0]-1], 'float')
    x = (y - intersept)/slope
    cv2.line(img, (int(x[0]), int(y[0])), (int(x[1]), int(y[1])), color, thickness)

In [ ]:
for index in range(len(images)):  
    img = images[index]['original']
    lines = images[index]['hough_lines']
    copy_img = np.copy(img)    
    right_lanes = []
    left_lanes = []    
    epsilon = 0.5
    middle_region_x = img.shape[1]/2
    for line in lines:
        x1,y1,x2,y2 = line[0]       
        if (x2-x1)!=0 and (y2-y1)!=0 :
            slope = (y2-y1)/(x2-x1)           
            if abs(slope) > epsilon:        
                if slope > 0 and middle_region_x<x1<x2:            
                    right_lanes.append([[x1,y1,x2,y2]])                   
                elif slope < 0 and x1<x2<middle_region_x:                    
                    left_lanes.append([[x1,y1,x2,y2]])
    slope, intersept = find_lane_lines_formula(right_lanes)
    color=[255, 0, 0]
    draw_full_lines(copy_img, slope, intersept, color, 10)
    slope, intersept = find_lane_lines_formula(left_lanes)
    color=[0, 0, 255]
    draw_full_lines(copy_img, slope, intersept, color, 10)   
    plt.figure(figsize=(10,8))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title('Original RGB')
    plt.subplot(1,2,2)
    plt.imshow(copy_img)
    plt.title('With full-bodied lanes')  
    images[index]['Left_Right_Colored_lines'] = copy_img
    plt.show()

Create a folder "The name you want" to store your test images output

In [ ]:
directory = 'Test_Images_Output'
for index, name_img in enumerate(os.listdir("Test_Images/")):
    saved_image = images[index]['Left_Right_Colored_lines']
    Image.fromarray(saved_image).save(os.path.join(directory,name_img))

In [ ]:
def process_image(image):   
    LAB_img = change_color_space(image,'LAB')
    yellow_lines = extract_line(LAB_img, [100,100,150], [220,180,255])   
    HLS_img = change_color_space(image,'HLS')
    white_lines = extract_line(HLS_img, [0,200,0], [180,255,255]) 
    white_lines = cv2.dilate(white_lines, kernel=np.ones((5,5), np.uint8), iterations=2)
    white_lines = cv2.erode(white_lines, kernel=np.ones((5,5), np.uint8), iterations=2)
    white_lines = cv2.dilate(white_lines, kernel=np.ones((5,5), np.uint8), iterations=1)    
    line_mask = yellow_lines + white_lines   
    masked_img = np.copy(image)
    masked_img = cv2.dilate(masked_img, kernel=np.ones((5,5), np.uint8), iterations=2)
    masked_img = cv2.erode(masked_img, kernel=np.ones((5,5), np.uint8), iterations=2)
    masked_img = cv2.dilate(masked_img, kernel=np.ones((5,5), np.uint8), iterations=1)
    masked_img[line_mask!=255] = [0,0,0]    
    cleaned_img = region_of_interest(masked_img)
    hough_img, lines = hough_lines(grayscale(cleaned_img), rho=1, theta=np.pi/180, threshold=17, min_line_len=7, max_line_gap=0)   
    overlay = np.zeros_like(image)   
    right_lanes = []
    left_lanes = []    
    epsilon = 0.5                        
    middle_region_x = image.shape[1]/2  
    for line in lines:
        x1,y1,x2,y2 = line[0]       
        if (x2-x1)!=0 and (y2-y1)!=0 :
            slope = (y2-y1)/(x2-x1)            
            if abs(slope) > epsilon:        
                if slope > 0 and middle_region_x<x1<x2:           
                    right_lanes.append([[x1,y1,x2,y2]])                    
                elif slope < 0 and x1<x2<middle_region_x:                  
                    left_lanes.append([[x1,y1,x2,y2]])   
    color=[0, 0, 255]
    if len(right_lanes)!=0:
        slope, intersept = find_lane_lines_formula(right_lanes)
        draw_full_lines(overlay, slope, intersept, color, 10)
    
    if len(left_lanes)!=0:
        slope, intersept = find_lane_lines_formula(left_lanes)
        draw_full_lines(overlay, slope, intersept, color, 10)  
    result = weighted_img(overlay, image, α=0.8, β=1., γ=0.)
    return result

In [ ]:
image = mpimg.imread('C:\\Users\\arnab\\Desktop\\PROJECTS\\Test_Images\\IMG_1.jpg')
result = process_image(image)
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
plt.imshow(image)
plt.title('Original RGB')
plt.subplot(1,2,2)
plt.imshow(result)
plt.title('With full-bodied overlaid lines')
plt.show()   

Create a folder "The name you want" to store test videos output

In [ ]:
output1 = 'C:\\Users\\arnab\\Desktop\\PROJECTS\\Test_Videos_Output\\VID_1.mp4'
clip1 = VideoFileClip('C:\\Users\\arnab\\Desktop\\PROJECTS\\Test_Videos\\VID_1.mp4')
vid1 = clip1.fl_image(process_image)
%time vid1.write_videofile(output1, audio=False)

In [ ]:
output1 = HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output1))

In [ ]:
output2 = 'C:\\Users\\arnab\\Desktop\\PROJECTS\\Test_Videos_Output\\VID_2(Right).mp4'
clip2 = VideoFileClip('C:\\Users\\arnab\\Desktop\\PROJECTS\\Test_Videos\\VID_2(Right).mp4')
vid2 = clip2.fl_image(process_image) 
%time vid2.write_videofile(output2, audio=False)z

In [ ]:
video_output = HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output2))

In [ ]:
output3 = 'C:\\Users\\arnab\\Desktop\\PROJECTS\\Test_Videos_Output\\VID_3(Left).mp4'
clip3 = VideoFileClip("C:\\Users\\arnab\\Desktop\\PROJECTS\\Test_videos\\VID_3(left).mp4")
vid3 = clip3.fl_image(process_image)
vid3.write_videofile(output3, audio=False, codec='mpeg4')

In [ ]:
video_output = HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output3))